Depending on the application, memory is the next component that will complete a chain. LangChain provides a ConversationalChain to track previous prompts and responses using the ConversationalBufferMemory class.

In [1]:
from dotenv import load_dotenv, dotenv_values
import os

# Load environment variables from .env file
config = dotenv_values("C:/Users/SACHENDRA/Documents/Activeloop/.env")
load_dotenv("C:/Users/SACHENDRA/Documents/Activeloop/.env")

True

In [2]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain import PromptTemplate, OpenAI, LLMChain
output_parser = CommaSeparatedListOutputParser()
template = """List all possible words as substitute for 'artificial' as comma separated."""

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

C:\Users\SACHENDRA\miniconda3\envs\activeloop2\Lib\site-packages\langchain\llms\openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\SACHENDRA\miniconda3\envs\activeloop2\Lib\site-packages\langchain\llms\openai.py:677: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

conversation.predict(input="List all possible words as substitute for 'artificial' as comma separated.")

'synthetic, simulated, man-made, fake, imitation'

In [4]:
# Now, we can ask it to return the following four replacement words. 
# It uses the memory to find the next options.

conversation.predict(input="And the next 4?")

'replica, faux, counterfeit, ersatz'

SEQUENTIAL CHAINS

In [5]:
# poet
poet_template: str = """You are an American poet, your job is to come up with\
poems based on a given theme.

Here is the theme you have been asked to generate a poem on:
{input}\
"""

poet_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["input"], template=poet_template)

# creating the poet chain
poet_chain: LLMChain = LLMChain(
    llm=llm, output_key="poem", prompt=poet_prompt_template)

# critic
critic_template: str = """You are a critic of poems, you are tasked\
to inspect the themes of poems. Identify whether the poem includes romantic expressions or descriptions of nature.

Your response should be in the following format, as a Python Dictionary.
poem: this should be the poem you received 
Romantic_expressions: True or False
Nature_descriptions: True or False

Here is the poem submitted to you:
{poem}\
"""

critic_prompt_template: PromptTemplate = PromptTemplate(
    input_variables=["poem"], template=critic_template)

# creating the critic chain
critic_chain: LLMChain = LLMChain(
    llm=llm, output_key="critic_verified", prompt=critic_prompt_template)

In this example we define two processes in a chain: one for generating poems based on a given theme ("poet") and another for evaluating these poems on their romantic and natural elements ("critic"). The poet process uses a template to instruct an AI model to create poems, while the critic process analyzes these poems, flagging them for specific content. The setup utilizes prompt templates and chains to seamlessly integrate content generation with content verification.

In [6]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[poet_chain, critic_chain])

In [7]:
# Run the poet and critic chain with a specific theme
theme: str = "the beauty of nature"
review = overall_chain.run(theme)

# Print the review to see the critic's evaluation
print(review)

{
"poem": "In the quiet of the morning light,\nNature's beauty shines so bright.\nThe sun rises over the horizon,\nPainting the sky with colors so fine.\n\nThe trees sway in the gentle breeze,\nTheir leaves rustling with such ease.\nBirds sing their sweet melodies,\nAs they flit and flutter through the trees.\n\nThe flowers bloom in vibrant hues,\nTheir petals kissed by morning dew.\nButterflies dance in the air,\nTheir delicate wings so fair.\n\nThe mountains stand tall and proud,\nTheir peaks reaching up to the clouds.\nRivers flow with a soothing sound,\nTheir waters clear and profound.\n\nNature's beauty is a sight to behold,\nA treasure more valuable than gold.\nIn every leaf and every flower,\nWe find peace and solace in nature's power.",
"Romantic_expressions": False,
"Nature_descriptions": True
}


Debug:

It is possible to trace the inner workings of any chain by setting the verbose argument to True. As you can see in the following code, the chain will return the initial prompt and the output. The output depends on the application. It may contain more information if there are more steps.

In [8]:
template = """List all possible words as substitute for 'artificial' as comma separated.

Current conversation:
{history}

{input}"""

conversation = ConversationChain(
    llm=llm,
    prompt=PromptTemplate(template=template, input_variables=["history", "input"], output_parser=output_parser),
    memory=ConversationBufferMemory(),
    verbose=True)

conversation.predict(input="")



> Entering new ConversationChain chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.

Current conversation:




> Finished chain.


'synthetic, man-made, fake, imitation, simulated, faux, fabricated, counterfeit, ersatz, pseudo'